In [ ]:
!pip install transformers
!pip install captum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.9 MB/s eta 0:00:00


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch import optim
import torchtext
from torchtext.vocab import GloVe
from torchtext.vocab import vocab
from torch.utils.data import random_split
import torch.nn.functional as F
from collections import Counter, OrderedDict
import pandas as pd
from tqdm import tqdm
from transformers import BertTokenizer, BertModel
import re
from captum.attr import IntegratedGradients, LayerIntegratedGradients

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ", device)

Device:  cuda


## Load the Data

In [ ]:
fake_news_df = pd.read_csv('Fake.csv', error_bad_lines=False)
fake_news_df = fake_news_df.drop(['subject', 'date'], axis=1)
fake_news_df['label'] = 0

<ipython-input-5-392e652f82ba>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  fake_news_df = pd.read_csv('Fake.csv', error_bad_lines=False)


In [ ]:
real_news_df = pd.read_csv('True.csv', error_bad_lines=False)
real_news_df = real_news_df.drop(['subject', 'date'], axis=1)
real_news_df['label'] = 1

<ipython-input-6-2abede05a107>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  real_news_df = pd.read_csv('True.csv', error_bad_lines=False)


In [ ]:
## For Fake.csv vs. True.csv
# all_data_df = pd.concat([fake_news_df, real_news_df], axis=0)
## For news_articles.csv
all_data_df = pd.read_csv('news_articles.csv', error_bad_lines=False)
all_data_df = all_data_df.drop(['author', 'published', 'language', 'site_url', 'main_img_url', 'type', 'title_without_stopwords', 'text_without_stopwords', 'hasImage'], axis=1)
all_data_df.dropna(axis=0, how='any', inplace=True)
all_data_df = all_data_df.replace({'Real': 1, 'Fake': 0})
non_string_rows = all_data_df['text'].apply(lambda x: not isinstance(x, str))
all_data_df = all_data_df.drop(index=all_data_df[non_string_rows].index)

## For both
fake_count = (all_data_df['label'] == 0).sum()
real_count = (all_data_df['label'] == 1).sum()
print("Fake articles: ", fake_count)
print("Real articles: ", real_count)
all_data_df

Fake articles:  1292
Real articles:  756


<ipython-input-7-55a8e611ce6a>:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  all_data_df = pd.read_csv('news_articles.csv', error_bad_lines=False)


,title,text,label
0,muslims busted they stole millions in govt ben...,print they should pay all the back all the mon...,1
1,re why did attorney general loretta lynch plea...,why did attorney general loretta lynch plead t...,1
2,breaking weiner cooperating with fbi on hillar...,red state \nfox news sunday reported this mor...,1
3,pin drop speech by father of daughter kidnappe...,email kayla mueller was a prisoner and torture...,1
4,fantastic trumps point plan to reform healthc...,email healthcare reform to make america great ...,1
...,...,...,...
2045,trump vs clinton a risk vs a disaster,check out hillarythemed haunted house anticlin...,1
2046,gingrich slutshames megyn kelly,good samaritan wearing indian headdress disarm...,1
2047,youtube bans clintons black son,skype sex scam a fortune built on shame moroc...,1
2048,wikileaks bombshells on hillary you need to know,posted by eddie while the skyhigh potency may ...,1


## Build the Vocab

In [ ]:
def clean_text(text):
    # This regular expression pattern will match the unnecessary text at the beginning of the text column
    pattern = r'^\s*[\w/,]*\s*\((Reuters|AP|AFP|CNN)\)\s*-\s*'
    cleaned_text = re.sub(pattern, '', text)
    # This regular expression pattern will match every instance of "Reuters" in the text, regardless of capitalization
    pattern = re.compile(r'reuters', re.IGNORECASE)
    cleaned_text = re.sub(pattern, '', cleaned_text)
    return cleaned_text

In [ ]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
all_data_df['title'] = all_data_df['title'].apply(lambda x: clean_text(x))
all_data_df['text'] = all_data_df['text'].apply(lambda x: clean_text(x))
all_data_df['title_tokens'] = all_data_df['title'].apply(tokenizer)
all_data_df['text_tokens'] = all_data_df['text'].apply(tokenizer)

In [ ]:
tokenized_data = all_data_df['title_tokens'].tolist() + all_data_df['text_tokens'].tolist()
token_counter = Counter(token for article_tokens in tokenized_data for token in article_tokens)
sorted_by_freq_tuples = sorted(token_counter.items(), key=lambda x: x[1], reverse=True)
ordered_dict = OrderedDict(sorted_by_freq_tuples)
built_vocab = vocab(ordered_dict)
word_to_index = built_vocab.get_stoi()
index_to_word = built_vocab.get_itos()
print("Vocab Size: ", len(built_vocab))

Vocab Size:  47740


In [ ]:
all_data_df['title_token_indices'] = all_data_df['title_tokens'].apply(lambda tokens: torch.LongTensor([word_to_index[token] for token in tokens]).to(device))
all_data_df['text_token_indices'] = all_data_df['text_tokens'].apply(lambda tokens: torch.LongTensor([word_to_index[token] for token in tokens]).to(device))
all_data_df

,title,text,label,title_tokens,text_tokens,title_token_indices,text_token_indices
0,muslims busted they stole millions in govt ben...,print they should pay all the back all the mon...,1,"[muslims, busted, they, stole, millions, in, g...","[print, they, should, pay, all, the, back, all...","[tensor(486, device='cuda:0'), tensor(5488, de...","[tensor(1090, device='cuda:0'), tensor(23, dev..."
1,re why did attorney general loretta lynch plea...,why did attorney general loretta lynch plead t...,1,"[re, why, did, attorney, general, loretta, lyn...","[why, did, attorney, general, loretta, lynch, ...","[tensor(3391, device='cuda:0'), tensor(136, de...","[tensor(136, device='cuda:0'), tensor(130, dev..."
2,breaking weiner cooperating with fbi on hillar...,red state \nfox news sunday reported this mor...,1,"[breaking, weiner, cooperating, with, fbi, on,...","[red, state, fox, news, sunday, reported, this...","[tensor(586, device='cuda:0'), tensor(1624, de...","[tensor(680, device='cuda:0'), tensor(85, devi..."
3,pin drop speech by father of daughter kidnappe...,email kayla mueller was a prisoner and torture...,1,"[pin, drop, speech, by, father, of, daughter, ...","[email, kayla, mueller, was, a, prisoner, and,...","[tensor(9858, device='cuda:0'), tensor(2929, d...","[tensor(115, device='cuda:0'), tensor(26908, d..."
4,fantastic trumps point plan to reform healthc...,email healthcare reform to make america great ...,1,"[fantastic, trumps, point, plan, to, reform, h...","[email, healthcare, reform, to, make, america,...","[tensor(5233, device='cuda:0'), tensor(169, de...","[tensor(115, device='cuda:0'), tensor(2157, de..."
...,...,...,...,...,...,...,...
2045,trump vs clinton a risk vs a disaster,check out hillarythemed haunted house anticlin...,1,"[trump, vs, clinton, a, risk, vs, a, disaster]","[check, out, hillarythemed, haunted, house, an...","[tensor(28, device='cuda:0'), tensor(2814, dev...","[tensor(1012, device='cuda:0'), tensor(56, dev..."
2046,gingrich slutshames megyn kelly,good samaritan wearing indian headdress disarm...,1,"[gingrich, slutshames, megyn, kelly]","[good, samaritan, wearing, indian, headdress, ...","[tensor(2387, device='cuda:0'), tensor(13929, ...","[tensor(165, device='cuda:0'), tensor(47704, d..."
2047,youtube bans clintons black son,skype sex scam a fortune built on shame moroc...,1,"[youtube, bans, clintons, black, son]","[skype, sex, scam, a, fortune, built, on, sham...","[tensor(819, device='cuda:0'), tensor(5998, de...","[tensor(47715, device='cuda:0'), tensor(581, d..."
2048,wikileaks bombshells on hillary you need to know,posted by eddie while the skyhigh potency may ...,1,"[wikileaks, bombshells, on, hillary, you, need...","[posted, by, eddie, while, the, skyhigh, poten...","[tensor(337, device='cuda:0'), tensor(13930, d...","[tensor(644, device='cuda:0'), tensor(15, devi..."


## Build the Dataset

In [ ]:
class NewsDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        raw_title = self.data.iloc[index]['title']
        raw_text = self.data.iloc[index]['text']
        title_tokens = self.data.iloc[index]['title_token_indices']
        text_tokens = self.data.iloc[index]['text_token_indices']
        label = self.data.iloc[index]['label']
        return raw_title, title_tokens, raw_text, text_tokens, label

In [ ]:
class CollateBatch:
    def __init__(self, padding_idx=0):
        self.padding_idx = padding_idx

    def __call__(self, batch):
        raw_title, title_tokens, raw_text, text_tokens, label = zip(*batch)
        padded_title_tokens = torch.nn.utils.rnn.pad_sequence(title_tokens, batch_first=True, padding_value=self.padding_idx).to(device)
        padded_text_tokens = torch.nn.utils.rnn.pad_sequence(text_tokens, batch_first=True, padding_value=self.padding_idx).to(device)
        label = torch.LongTensor(label)
        return raw_title, padded_title_tokens, raw_text, padded_text_tokens, label

In [ ]:
dataset = NewsDataset(all_data_df)
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

## Setup Model Training and Evaluation

In [ ]:
batch_size = 32
collate_fn = CollateBatch()
train_loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=collate_fn)

In [ ]:
def eval_model(model, data, loss_fn, use_raw_data=False):
    model.eval()
    total_loss = 0
    correct_predictions = 0
    one_predictions = 0
    zero_predictions = 0

    with torch.no_grad():
        for batch_idx, batch in tqdm(enumerate(data), total=len(data)):
            if use_raw_data:
              titles, _, text, _, labels = batch
            else:
              _, titles, _, text, labels = batch
              titles = titles.to(device)
              text = text.to(device)
            labels = labels.to(device)

            outs = model.forward(title=titles, text=text).to(device)
            predictions = torch.argmax(outs, dim=-1).to(device)
            one_predictions += torch.sum(predictions == 1)
            zero_predictions += torch.sum(predictions == 0)
            loss = loss_fn(outs, labels)
            total_loss += loss.item()
            correct_predictions += torch.sum(predictions == labels)

    print("One: ", one_predictions)
    print("Zero: ", zero_predictions)
    epoch_loss = total_loss / len(data)
    accuracy = correct_predictions / len(data.dataset)
    return epoch_loss, accuracy

def train_epoch(model, data, loss_fn, optimizer, use_raw_data=False):
    total_loss = 0
    correct_predictions = 0
    model.train()
    for batch_idx, batch in tqdm(enumerate(data), total=len(data)):
        if use_raw_data:
          titles, _, text, _, labels = batch
        else:
          _, titles, _, text, labels = batch
          titles = titles.to(device)
          text = text.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outs = model.forward(title=titles, text=text).to(device)
        predictions = torch.argmax(outs, dim=-1).to(device)
        loss = loss_fn(outs, labels)
        total_loss += loss.item()
        correct_predictions += torch.sum(predictions == labels)
        loss.backward()
        optimizer.step()

    epoch_loss = total_loss / len(data)
    accuracy = correct_predictions / len(data.dataset)
    return epoch_loss, accuracy

def train_model(model, num_epochs, train_data, val_data, loss_fn, optimizer, use_raw_data=False):
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        train_loss, train_acc = train_epoch(model, train_data, loss_fn, optimizer, use_raw_data)
        print(f"Train loss: {train_loss:.4f}, Train accuracy: {train_acc:.4f}")

        val_loss, val_acc = eval_model(model, val_data, loss_fn, use_raw_data)
        print(f"Validation loss: {val_loss:.4f}, Validation accuracy: {val_acc:.4f}")

In [ ]:
def evaluate_model_features(model, path_to_saved_model_weights, use_raw_data=False):
    model.load_state_dict(torch.load(path_to_saved_model_weights))
    ig = LayerIntegratedGradients(model, model.embedding_layer)
    score_dict = dict()
    for batch_idx, batch in tqdm(enumerate(test_loader), total=len(test_loader)):
      if use_raw_data:
        return None, None
      else:
        _, title_indices, _, text_indices, _ = batch
        title_indices = title_indices.to(device)
        text_indices = text_indices.to(device)
      input_indices = (title_indices, text_indices)
      attr_scores = ig.attribute(inputs=input_indices, target=1, internal_batch_size=32)
      word_scores = torch.abs(attr_scores.sum(axis=2))
      word_scores /= word_scores.sum()
      word_scores_list = word_scores.tolist()[0]
      input_text_list = all_data_df.iloc[0]['text_tokens']
      attribution_dict = dict(zip(input_text_list, word_scores_list))
      for word, score in attribution_dict.items():
        if word not in score_dict:
          score_dict[word] = 0
        score_dict[word] += score
    for word, score in score_dict.items():
      score_dict[word] = score_dict[word] / len(test_loader.dataset)
    top_true_features = sorted(score_dict.items(), key=lambda x: x[1], reverse=True)
    top_true_features = top_true_features[:10]
    top_false_features = sorted(score_dict.items(), key=lambda x: x[1], reverse=False)
    top_false_features = top_false_features[:10]
    return top_true_features, top_false_features

## Build, Train, and Test NewsPretrainedBertModel

In [ ]:
class NewsPretrainedBertModel(nn.Module):
    def __init__(self):
        super(NewsPretrainedBertModel, self).__init__()
        self.tokenizer = BertTokenizer.from_pretrained("prajjwal1/bert-tiny")
        self.max_len = 512
        self.bert = BertModel.from_pretrained("prajjwal1/bert-tiny")
        for param in self.bert.embeddings.parameters():
            param.requires_grad = False
        self.dropout = nn.Dropout(0.3)
        self.linear = nn.Linear(self.bert.config.hidden_size, 2)

    def forward(self, title, text):
        title_and_text = [a + ' ' + b for a, b in zip(title, text)]
        encoding = self.tokenizer.batch_encode_plus(
            title_and_text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        input_ids = encoding["input_ids"].to(device)
        attention_mask = encoding["attention_mask"].to(device)
        embeddings = self.bert(input_ids=input_ids, attention_mask=attention_mask)[1].to(device)
        output = self.dropout(embeddings).to(device)
        output = self.linear(output).to(device)
        return output

In [ ]:
model = NewsPretrainedBertModel().to(device)
print(model)
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=5e-4)
train_model(model, 18, train_loader, val_loader, loss_fn, optimizer, use_raw_data=True)

NewsPretrainedBertModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-12, element

100%|██████████| 45/45 [00:26<00:00,  1.68it/s]


Train loss: 0.6678, Train accuracy: 0.5897


100%|██████████| 10/10 [00:03<00:00,  2.61it/s]


One:  tensor(137, device='cuda:0')
Zero:  tensor(170, device='cuda:0')
Validation loss: 0.6261, Validation accuracy: 0.5896
Epoch 2/18


...

One:  tensor(102, device='cuda:0')
Zero:  tensor(205, device='cuda:0')
Validation loss: 0.8101, Validation accuracy: 0.7296
Epoch 18/18


100%|██████████| 45/45 [00:20<00:00,  2.16it/s]


Train loss: 0.2174, Train accuracy: 0.9142


100%|██████████| 10/10 [00:04<00:00,  2.19it/s]

One:  tensor(94, device='cuda:0')
Zero:  tensor(213, device='cuda:0')
Validation loss: 0.7328, Validation accuracy: 0.7166


In [ ]:
test_loss, test_acc = eval_model(model, test_loader, loss_fn, use_raw_data=True)
print(f"Test loss: {test_loss:.4f}, Test accuracy: {test_acc:.4f}")

100%|██████████| 10/10 [00:05<00:00,  2.00it/s]

One:  tensor(100, device='cuda:0')
Zero:  tensor(208, device='cuda:0')
Test loss: 0.6674, Test accuracy: 0.7045


## Build, Train, and Test NewsCNNModel

In [ ]:
class NewsCNNModel(nn.Module):
    def __init__(self, num_filters, filter_sizes, dropout, output_dim=2, embedding_dim=300):
        super(NewsCNNModel, self).__init__()
        vocab_size = len(built_vocab)
        embedding_weights = self.get_embedding_weights(vocab_size, embedding_dim)
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
        self.embedding_layer.weight.data.copy_(embedding_weights)
        self.convs = nn.ModuleList([
            nn.Conv2d(in_channels=1, out_channels=num_filters, kernel_size=(filter_size, embedding_dim))
            for filter_size in filter_sizes
        ])
        self.dropout = nn.Dropout(dropout)
        self.prediction_layer = nn.Linear(num_filters * len(filter_sizes), output_dim)

    def get_embedding_weights(self, vocab_size, embedding_dim):
        glove_embeddings = GloVe(name='6B', dim=embedding_dim)
        embedding_weights = []
        for i in range(vocab_size):
            word = index_to_word[i]
            if word in glove_embeddings.stoi:
                embedding_weights.append(glove_embeddings.vectors[glove_embeddings.stoi[word]])
            else:
                embedding_weights.append(torch.randn(embedding_dim))
        embedding_weights = torch.stack(embedding_weights)
        return embedding_weights

    def forward(self, title, text):
        title_embedded = self.embedding_layer(title.long())
        text_embedded = self.embedding_layer(text.long())
        embedding = torch.cat((title_embedded, text_embedded), dim=1)
        embedding = embedding.unsqueeze(1)
        conv_results = [F.relu(conv(embedding)).squeeze(3) for conv in self.convs]
        pooled_results = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conv_results]
        cat = torch.cat(pooled_results, dim=-1)
        out = self.dropout(cat)
        out = self.prediction_layer(cat)
        return out


In [ ]:
NUM_FILTERS = 10
FILTER_SIZES = [3, 4, 5]
DROPOUT = 0.6
model = NewsCNNModel(num_filters=NUM_FILTERS, filter_sizes=FILTER_SIZES, dropout=DROPOUT, embedding_dim=100).to(device)
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=5e-4)
train_model(model, 10, train_loader, val_loader, loss_fn, optimizer)

.vector_cache/glove.6B.zip: 862MB [02:50, 5.07MB/s]                           
100%|█████████▉| 399999/400000 [00:20<00:00, 19505.78it/s]


Epoch 1/10


100%|██████████| 45/45 [00:07<00:00,  5.74it/s]


Train loss: 0.6822, Train accuracy: 0.5736


100%|██████████| 10/10 [00:00<00:00, 43.78it/s]


...

One:  tensor(73, device='cuda:0')
Zero:  tensor(234, device='cuda:0')
Validation loss: 0.5181, Validation accuracy: 0.7329
Epoch 10/10


100%|██████████| 45/45 [00:03<00:00, 14.20it/s]


Train loss: 0.1617, Train accuracy: 0.9944


100%|██████████| 10/10 [00:00<00:00, 44.03it/s]

One:  tensor(74, device='cuda:0')
Zero:  tensor(233, device='cuda:0')
Validation loss: 0.5113, Validation accuracy: 0.7362


In [ ]:
test_loss, test_acc = eval_model(model, test_loader, loss_fn)
print(f"Test loss: {test_loss:.4f}, Test accuracy: {test_acc:.4f}")

100%|██████████| 10/10 [00:00<00:00, 44.67it/s]

One:  tensor(83, device='cuda:0')
Zero:  tensor(225, device='cuda:0')
Test loss: 0.4602, Test accuracy: 0.7922


In [ ]:
torch.save(model.state_dict(), 'news_cnn_model.pth')
true_features, false_features = evaluate_model_features(model, 'news_cnn_model.pth')
print("\nTop True Features: ")
for word, score in true_features:
  print(str(word) + ': ' + str(score))

print("Top False Features: ")
for word, score in false_features:
  print(str(word) + ': ' + str(score))

100%|██████████| 10/10 [00:16<00:00,  1.68s/it]


Top True Features: 
stole: 4.527533589262679e-05
government: 1.7380693690668997e-05
from: 1.0525383772364673e-05
pay: 1.0035579295704103e-05
print: 7.3820487760564434e-06
four: 6.769105839194586e-06
did: 6.70608717099143e-06
one: 6.01180432688282e-06
cases: 5.5986239152507375e-06
why: 5.422659492654506e-06
Top False Features: 
entire: 0.0
to: 0.0
group: 0.0
everyone: 9.718516963871176e-08
asap: 1.1012509919290378e-07
family: 1.136343130579054e-07
bust: 1.334110453013289e-07
who: 2.016055994087126e-07
commit: 2.067044073150492e-07
weve: 3.2405505470427127e-07


## Build, Train, and Test NewsLSTMModel

In [ ]:
class NewsLSTMModel(nn.Module):
    def __init__(self, hidden_dim, dropout, embedding_dim=300, output_dim=2):
        super(NewsLSTMModel, self).__init__()
        vocab_size = len(built_vocab)
        embedding_weights = self.get_embedding_weights(vocab_size, embedding_dim)
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
        self.embedding_layer.weight.data.copy_(embedding_weights)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True, dropout=dropout)
        self.fc1 = nn.Linear(hidden_dim * 2, output_dim)

    def get_embedding_weights(self, vocab_size, embedding_dim):
        glove_embeddings = GloVe(name='6B', dim=embedding_dim)
        embedding_weights = []
        for i in range(vocab_size):
            word = index_to_word[i]
            if word in glove_embeddings.stoi:
                embedding_weights.append(glove_embeddings.vectors[glove_embeddings.stoi[word]])
            else:
                embedding_weights.append(torch.randn(embedding_dim))
        embedding_weights = torch.stack(embedding_weights)
        return embedding_weights

    def forward(self, title, text):
        title_embedded = self.embedding_layer(title)
        text_embedded = self.embedding_layer(text)
        cat = torch.cat((title_embedded, text_embedded), dim=1)
        outs, _ = self.rnn(cat)
        outs = torch.max(outs, dim=1)[0]
        logits = self.fc1(outs)
        return logits


In [ ]:
HIDDEN_DIM = 100
DROPOUT = 0.6
model = NewsLSTMModel(hidden_dim=HIDDEN_DIM, dropout=DROPOUT).to(device)
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
train_model(model, 15, train_loader, val_loader, loss_fn, optimizer)

100%|█████████▉| 399999/400000 [00:57<00:00, 6908.42it/s]
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.6 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch 1/15


100%|██████████| 45/45 [00:05<00:00,  7.69it/s]


Train loss: 0.6467, Train accuracy: 0.6169


100%|██████████| 10/10 [00:00<00:00, 22.32it/s]


...

One:  tensor(91, device='cuda:0')
Zero:  tensor(216, device='cuda:0')
Validation loss: 0.5714, Validation accuracy: 0.7785
Epoch 15/15


100%|██████████| 45/45 [00:04<00:00,  9.38it/s]


Train loss: 0.0070, Train accuracy: 0.9986


100%|██████████| 10/10 [00:00<00:00, 21.55it/s]

One:  tensor(83, device='cuda:0')
Zero:  tensor(224, device='cuda:0')
Validation loss: 0.5123, Validation accuracy: 0.8046


In [ ]:
test_loss, test_acc = eval_model(model, test_loader, loss_fn)
print(f"Test loss: {test_loss:.4f}, Test accuracy: {test_acc:.4f}")

100%|██████████| 10/10 [00:00<00:00, 24.01it/s]

One:  tensor(101, device='cuda:0')
Zero:  tensor(207, device='cuda:0')
Test loss: 0.3941, Test accuracy: 0.8377


In [ ]:
torch.backends.cudnn.enabled=False
torch.save(model.state_dict(), 'news_lstm_model.pth')
true_features, false_features = evaluate_model_features(model, 'news_lstm_model.pth')
print("\nTop True Features: ")
for word, score in true_features:
  print(str(word) + ': ' + str(score))

print("Top False Features: ")
for word, score in false_features:
  print(str(word) + ': ' + str(score))

  0%|          | 0/10 [01:46<?, ?it/s]


KeyboardInterrupt: ignored

## Build, Train, and Test NewsBertCNNModel

In [ ]:
class NewsBertCNNModel(nn.Module):
    def __init__(self, num_filters, filter_sizes, dropout, output_dim=2):
        super(NewsBertCNNModel, self).__init__()
        self.tokenizer = BertTokenizer.from_pretrained("prajjwal1/bert-tiny")
        self.max_len = 512
        self.bert = BertModel.from_pretrained("prajjwal1/bert-tiny")
        for param in self.bert.embeddings.parameters():
            param.requires_grad = False
        self.dropout = nn.Dropout(dropout)
        self.convs = nn.ModuleList([
            nn.Conv2d(in_channels=1, out_channels=num_filters, kernel_size=(filter_size, self.bert.config.hidden_size))
            for filter_size in filter_sizes
        ])
        self.dropout_second = nn.Dropout(dropout)
        self.prediction_layer = nn.Linear(num_filters * len(filter_sizes), output_dim)

    def forward(self, title, text):
        title_and_text = [a + ' ' + b for a, b in zip(title, text)]
        encoding = self.tokenizer.batch_encode_plus(
            title_and_text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        input_ids = encoding["input_ids"].to(device)
        attention_mask = encoding["attention_mask"].to(device)
        text_embedding = self.bert(input_ids=input_ids, attention_mask=attention_mask)[0].to(device)
        text_embedding = self.dropout(text_embedding).to(device)
        text_embedding = text_embedding.unsqueeze(1)
        conv_results = [F.relu(conv(text_embedding)).squeeze(3) for conv in self.convs]
        pooled_results = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conv_results]
        cat = torch.cat(pooled_results, dim=-1)
        out = self.dropout_second(cat)
        out = self.prediction_layer(cat)
        return out

In [ ]:
NUM_FILTERS = 100
FILTER_SIZES = [3, 4, 5]
OUTPUT_DIM = 2
DROPOUT = 0.5
model = NewsBertCNNModel(NUM_FILTERS, FILTER_SIZES, DROPOUT).to(device)
print(model)
# loss_fn = nn.CrossEntropyLoss().to(device)
# optimizer = optim.Adam(model.parameters(), lr=1e-3)
# train_model(model, 15, train_loader, val_loader, loss_fn, optimizer, use_raw_data=True)

NewsBertCNNModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_af

In [ ]:
test_loss, test_acc = eval_model(model, test_loader, loss_fn, use_raw_data=True)
print(f"Test loss: {test_loss:.4f}, Test accuracy: {test_acc:.4f}")

100%|██████████| 10/10 [00:04<00:00,  2.36it/s]

One:  tensor(308, device='cuda:0')
Zero:  tensor(0, device='cuda:0')
Test loss: 0.8933, Test accuracy: 0.3604


## Build, Train, and Test NewsBertLSTMModel

In [ ]:
from unicodedata import bidirectional
class NewsBertLSTMModel(nn.Module):
    def __init__(self, hidden_dim, dropout, output_dim=2):
        super(NewsBertLSTMModel, self).__init__()
        self.tokenizer = BertTokenizer.from_pretrained("prajjwal1/bert-tiny")
        self.max_len = 512
        self.bert = BertModel.from_pretrained("prajjwal1/bert-tiny")
        for param in self.bert.embeddings.parameters():
            param.requires_grad = False
        self.dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(self.bert.config.hidden_size, hidden_dim, batch_first=True, dropout=dropout, bidirectional=True)
        self.prediction_layer = nn.Linear(2 * hidden_dim, output_dim)

    def forward(self, title, text):
        title_and_text = [a + ' ' + b for a, b in zip(title, text)]
        encoding = self.tokenizer.batch_encode_plus(
            title_and_text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        input_ids = encoding["input_ids"].to(device)
        attention_mask = encoding["attention_mask"].to(device)
        text_embedding = self.bert(input_ids=input_ids, attention_mask=attention_mask)[0].to(device)
        text_embedding = self.dropout(text_embedding).to(device)
        lstm_output, _ = self.lstm(text_embedding)
        lstm_output = torch.max(lstm_output, dim=1)[0]
        out = self.prediction_layer(lstm_output)
        return out


In [ ]:
HIDDEN_DIM = 100
DROPOUT = 0.5
model = NewsBertLSTMModel(hidden_dim=HIDDEN_DIM, dropout=DROPOUT).to(device)
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-5)
train_model(model, 5, train_loader, val_loader, loss_fn, optimizer, use_raw_data=True)

In [ ]:
test_loss, test_acc = eval_model(model, test_loader, loss_fn, use_raw_data=True)
print(f"Test loss: {test_loss:.4f}, Test accuracy: {test_acc:.4f}")

## Build, Train, and Test NewsCNNLSTMModel

In [ ]:
class NewsCNNLSTMModel(nn.Module):
    def __init__(self, num_filters, filter_sizes, hidden_dim, dropout, output_dim=2, embedding_dim=300):
        super(NewsCNNLSTMModel, self).__init__()
        vocab_size = len(built_vocab)
        embedding_weights = self.get_embedding_weights(vocab_size, embedding_dim)
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
        self.embedding_layer.weight.data.copy_(embedding_weights)
        self.convs = nn.ModuleList([
            nn.Conv2d(in_channels=1, out_channels=num_filters, kernel_size=(filter_size, embedding_dim))
            for filter_size in filter_sizes
        ])
        self.lstm = nn.LSTM(input_size=num_filters * len(filter_sizes), hidden_size=hidden_dim, batch_first=True, bidirectional=True, dropout=dropout)
        self.prediction_layer = nn.Linear(2 * hidden_dim, output_dim)

    def get_embedding_weights(self, vocab_size, embedding_dim):
        glove_embeddings = GloVe(name='6B', dim=embedding_dim)
        embedding_weights = []
        for i in range(vocab_size):
            word = index_to_word[i]
            if word in glove_embeddings.stoi:
                embedding_weights.append(glove_embeddings.vectors[glove_embeddings.stoi[word]])
            else:
                embedding_weights.append(torch.randn(embedding_dim))
        embedding_weights = torch.stack(embedding_weights)
        return embedding_weights

    def forward(self, title, text):
        title_embedded = self.embedding_layer(title)
        text_embedded = self.embedding_layer(text)
        embedding = torch.cat((title_embedded, text_embedded), dim=1)
        embedding = embedding.unsqueeze(1)
        conv_results = [F.relu(conv(embedding)).squeeze(3) for conv in self.convs]
        pooled_results = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conv_results]
        cat = torch.cat(pooled_results, dim=-1)
        lstm_out, _ = self.lstm(cat)
        outs = self.prediction_layer(lstm_out)
        return outs

In [ ]:
NUM_FILTERS = 10
FILTER_SIZES = [3, 4, 5]
HIDDEN_DIM = 100
DROPOUT = 0.6
model = NewsCNNLSTMModel(num_filters=NUM_FILTERS, filter_sizes=FILTER_SIZES, hidden_dim=HIDDEN_DIM, dropout=DROPOUT).to(device)
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=5e-4)
train_model(model, 15, train_loader, val_loader, loss_fn, optimizer)

Epoch 1/15


100%|██████████| 45/45 [00:14<00:00,  3.10it/s]


Train loss: 0.6620, Train accuracy: 0.6183


100%|██████████| 10/10 [00:01<00:00,  6.62it/s]


One:  tensor(0, device='cuda:0')
Zero:  tensor(307, device='cuda:0')
Validation loss: 0.6440, Validation accuracy: 0.6775
Epoch 2/15


...

One:  tensor(93, device='cuda:0')
Zero:  tensor(214, device='cuda:0')
Validation loss: 0.8905, Validation accuracy: 0.7264
Epoch 15/15


100%|██████████| 45/45 [00:14<00:00,  3.20it/s]


Train loss: 0.0007, Train accuracy: 1.0000


100%|██████████| 10/10 [00:01<00:00,  7.28it/s]

One:  tensor(96, device='cuda:0')
Zero:  tensor(211, device='cuda:0')
Validation loss: 0.9004, Validation accuracy: 0.7231


In [ ]:
test_loss, test_acc = eval_model(model, test_loader, loss_fn)
print(f"Test loss: {test_loss:.4f}, Test accuracy: {test_acc:.4f}")

100%|██████████| 10/10 [00:01<00:00,  7.43it/s]

One:  tensor(108, device='cuda:0')
Zero:  tensor(200, device='cuda:0')
Test loss: 0.6654, Test accuracy: 0.7565


In [ ]:
torch.backends.cudnn.enabled=False
torch.save(model.state_dict(), 'news_cnn_lstm_model.pth')
true_features, false_features = evaluate_model_features(model, 'news_cnn_lstm_model.pth')
print("\nTop True Features: ")
for word, score in true_features:
  print(str(word) + ': ' + str(score))

print("Top False Features: ")
for word, score in false_features:
  print(str(word) + ': ' + str(score))

100%|██████████| 10/10 [01:53<00:00, 11.36s/it]


Top True Features: 
print: 3.3059271474049745e-05
pay: 1.3497917818197894e-05
they: 1.0831680625378735e-05
should: 1.0275291545175875e-05
everyone: 9.781218068055504e-06
control: 9.519836190425666e-06
did: 8.554549664184063e-06
entire: 8.281229717616209e-06
stole: 7.3423176914480846e-06
family: 7.342038291567793e-06
Top False Features: 
the: 0.0
to: 0.0
years: 0.0
another: 0.0
somalis: 0.0
this: 0.0
one: 0.0
where: 0.0
taxpayers: 7.136245616094858e-08
again: 1.1137720924149857e-07


## Build, Train, and Test NewsBertCNNLSTMModel

In [ ]:
class NewsBertCNNLSTMModel(nn.Module):
    def __init__(self, num_filters, filter_sizes, hidden_dim, dropout, output_dim=2):
        super(NewsBertCNNLSTMModel, self).__init__()
        self.tokenizer = BertTokenizer.from_pretrained("prajjwal1/bert-tiny")
        self.max_len = 512
        self.bert = BertModel.from_pretrained("prajjwal1/bert-tiny")
        for param in self.bert.embeddings.parameters():
            param.requires_grad = False
        self.dropout = nn.Dropout(dropout)
        self.convs = nn.ModuleList([
            nn.Conv2d(in_channels=1, out_channels=num_filters, kernel_size=(filter_size, self.bert.config.hidden_size))
            for filter_size in filter_sizes
        ])
        self.lstm = nn.LSTM(input_size=num_filters * len(filter_sizes), hidden_size=hidden_dim, batch_first=True, bidirectional=True, dropout=dropout)
        self.prediction_layer = nn.Linear(2 * hidden_dim, output_dim)

    def forward(self, title, text):
        title_and_text = [a + ' ' + b for a, b in zip(title, text)]
        encoding = self.tokenizer.batch_encode_plus(
            title_and_text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        input_ids = encoding["input_ids"].to(device)
        attention_mask = encoding["attention_mask"].to(device)
        text_embedding = self.bert(input_ids=input_ids, attention_mask=attention_mask)[0].to(device)
        text_embedding = self.dropout(text_embedding).to(device)
        text_embedding = text_embedding.unsqueeze(1)
        conv_results = [F.relu(conv(text_embedding)).squeeze(3) for conv in self.convs]
        pooled_results = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conv_results]
        cat = torch.cat(pooled_results, dim=-1)
        lstm_out, _ = self.lstm(cat)
        outs = self.prediction_layer(lstm_out)
        return outs

In [ ]:
NUM_FILTERS = 100
FILTER_SIZES = [3, 4, 5]
HIDDEN_DIM = 100
DROPOUT = 0.5
model = NewsBertCNNLSTMModel(num_filters=NUM_FILTERS, filter_sizes=FILTER_SIZES, hidden_dim=HIDDEN_DIM, dropout=0.5).to(device)
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-5)
train_model(model, 5, train_loader, val_loader, loss_fn, optimizer, use_raw_data=True)

In [ ]:
test_loss, test_acc = eval_model(model, test_loader, loss_fn, use_raw_data=True)
print(f"Test loss: {test_loss:.4f}, Test accuracy: {test_acc:.4f}")

##Linear Model

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from scipy.sparse import hstack

# Create vectorizer and fit it to the entire dataset
vectorizer = CountVectorizer()
# all_data_df['title'] + ' ' +
features = vectorizer.fit_transform(all_data_df['text'])

train_size = int(0.8 * len(all_data_df))
train_features, train_labels = features[:train_size], all_data_df['label'][:train_size]
test_features, test_labels = features[train_size:], all_data_df['label'][train_size:]

# Fit logistic regression model to the training data
clf = LogisticRegression(max_iter=1000)
clf.fit(train_features, train_labels)

# Evaluate the model on the test data
pred_labels = clf.predict(test_features)
accuracy = accuracy_score(test_labels, pred_labels)
print(f"Accuracy: {accuracy}")

In [ ]:
# Get feature names and coefficients
feature_names = np.array(vectorizer.get_feature_names_out())
coefficients = clf.coef_[0]

# Get top feature indices and their coefficients for title
top_indices = coefficients.argsort()[-20:][::-1]
top_features = feature_names[top_indices]
top_coefficients = coefficients[top_indices]

print("Top Features:")
i = 0
for feature_name in top_features:
  coefficient = top_coefficients[i]
  print(f"{feature_name}: {coefficient}")
  i += 1

In [ ]:
# Get bottom feature indices and their coefficients for title
bottom_indices = coefficients.argsort()[:20]
bottom_features = feature_names[bottom_indices]
bottom_coefficients = coefficients[bottom_indices]
print("Bottom Features:")
i = 0
for feature_name in bottom_features:
  coefficient = bottom_coefficients[i]
  print(f"{feature_name}: {coefficient}")
  i += 1